# DATA TRANSFORMATION - 19_consolidacion
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [2]:
dcc = gf.diccionario_llaves()

In [3]:
mods = ['Hospitalizaciones', 'Urgencias', 'Farmacovigilancia RAM']

In [4]:
# execute initial file
files = 'C:/Users/monic/documentos/ds4a/ds4a_team80/1_data_transformation/1_consolidacion_mensual/'
file_0 = files + '0_initial.py'

exec(open(file_0).read())

In [5]:
for m in mods :
    file_i = files + dcc[m]['prefi'] + '.py'
    exec(open(file_i).read())


<string>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
base = base_final_urg
base.shape

(52320, 7)

In [7]:
base = base_final_hos.merge(base_final_urg, how = 'left')
base.shape

(52320, 15)

In [8]:
base = base.merge(base_final_far, how = 'left')
base.shape

(52320, 26)

In [9]:
base.head()

,id,year,month,hos_num,hos_num_j,hos_est,hos_est_j,hos_uci,hos_uci_j,hos_uce,...,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
basex = base.head()
basex


,id,year,month,hos_num,hos_num_j,hos_est,hos_est_j,hos_uci,hos_uci_j,hos_uce,...,far_rea_total,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
base['nuls'] = base.isnull().sum(axis = 1)
base.shape

(52320, 27)

In [16]:
base.groupby('nuls')['id'].count().reset_index(name='cuenta').sort_values('nuls')

,nuls,cuenta
0,8,1
1,11,188
2,12,4
3,15,388
4,19,1870
5,23,49869


In [23]:
base.shape[1]

27

In [28]:
todo_null = base.shape[1] - 4    #id, month, year, nuls
base_ = base[base['nuls'] < todo_null ]
base_.reset_index( inplace = True )
base_.shape

(2451, 28)

In [29]:
base_[ids].head()

,id,year,month
0,500547,2017,2
1,500547,2018,12
2,500547,2019,1
3,500547,2019,6
4,500547,2020,2


,index,id,year,month,hos_num,hos_num_j,hos_est,hos_est_j,hos_uci,hos_uci_j,...,far_evo_aun,far_evo_con,far_des_no,far_des_si,far_mis_si,far_cau_def,far_cau_otra,far_gra_leve,far_gra_mod,nuls
0,13,500547,2017,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,35,500547,2018,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
2,36,500547,2019,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
3,41,500547,2019,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
4,49,500547,2020,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
